<a href="https://colab.research.google.com/github/palarunava/machine-learning-courses/blob/main/machine-learning-specialization/supervised-ml-regression-classification/C1_W2_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Get the data**

In [85]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/housing-prices-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'housing-prices-dataset' dataset.
Path to dataset files: /kaggle/input/housing-prices-dataset


In [86]:
import pandas as pd
import numpy as np

In [87]:
data = pd.read_csv(path + "/Housing.csv")

In [88]:
data['mainroad'] = data['mainroad'].map({'yes': 1, 'no': 0})
data['guestroom'] = data['guestroom'].map({'yes': 1, 'no': 0})
data['basement'] = data['basement'].map({'yes': 1, 'no': 0})
data['hotwaterheating'] = data['hotwaterheating'].map({'yes': 1, 'no': 0})
data['airconditioning'] = data['airconditioning'].map({'yes': 1, 'no': 0})
data['prefarea'] = data['prefarea'].map({'yes': 1, 'no': 0})
data['furnishingstatus'] = data['furnishingstatus'].map({'furnished': 2, 'semi-furnished': 1, 'unfurnished': 0})

In [89]:
data

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,0
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,1
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,0
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,2


In [90]:
import math
num_samples = data.shape[0]
train_valid_test_split_ratio = [75, 20, 5] # 75:20:5 split
train_num_samples = math.floor((num_samples / 100) * train_valid_test_split_ratio[0])
valid_num_samples = math.floor((num_samples / 100) * train_valid_test_split_ratio[1])
test_num_samples = num_samples - train_num_samples - valid_num_samples
print(f"Number of training samples: {train_num_samples}")
print(f"Number of validation samples: {valid_num_samples}")
print(f"Number of testing samples: {test_num_samples}")

Number of training samples: 408
Number of validation samples: 109
Number of testing samples: 28


In [91]:
data_train = data.iloc[:train_num_samples, :]
data_valid = data.iloc[train_num_samples:(train_num_samples + valid_num_samples), :]
data_test = data.iloc[(train_num_samples + valid_num_samples):, :]

In [92]:
X_train = data_train.iloc[:, 1:]
y_train = data_train.iloc[:, 0]
assert X_train.shape[0] == y_train.shape[0]
print("Information about the data:")
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")
print("\n")
X_train.info()

Information about the data:
Number of training samples: 408
Number of features: 12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   area              408 non-null    int64
 1   bedrooms          408 non-null    int64
 2   bathrooms         408 non-null    int64
 3   stories           408 non-null    int64
 4   mainroad          408 non-null    int64
 5   guestroom         408 non-null    int64
 6   basement          408 non-null    int64
 7   hotwaterheating   408 non-null    int64
 8   airconditioning   408 non-null    int64
 9   parking           408 non-null    int64
 10  prefarea          408 non-null    int64
 11  furnishingstatus  408 non-null    int64
dtypes: int64(12)
memory usage: 38.4 KB


Normalize the data - Z score

In [93]:
X_norm = (X_train - X_train.mean()) / X_train.std()
#X_norm.to_numpy()
X_norm

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,0.815896,1.296780,1.170178,1.165895,0.296284,-0.531343,-0.823012,-0.238443,1.243463,1.318835,1.538209,1.290881
1,1.511957,1.296780,4.871204,2.249460,0.296284,-0.531343,-0.823012,-0.238443,1.243463,2.435193,-0.648513,1.290881
2,1.963945,-0.106632,1.170178,0.082330,0.296284,-0.531343,1.212072,-0.238443,-0.802234,1.318835,1.538209,-0.113598
3,0.852055,1.296780,1.170178,0.082330,0.296284,-0.531343,1.212072,-0.238443,1.243463,2.435193,1.538209,1.290881
4,0.815896,1.296780,-0.680336,0.082330,0.296284,1.877411,1.212072,-0.238443,1.243463,1.318835,-0.648513,1.290881
...,...,...,...,...,...,...,...,...,...,...,...,...
403,3.312677,-0.106632,-0.680336,-1.001235,0.296284,-0.531343,-0.823012,-0.238443,-0.802234,-0.913881,-0.648513,-1.518076
404,-0.323114,-0.106632,-0.680336,0.082330,-3.366866,-0.531343,-0.823012,-0.238443,-0.802234,-0.913881,-0.648513,-1.518076
405,-1.154771,-0.106632,-0.680336,-1.001235,0.296284,-0.531343,-0.823012,-0.238443,-0.802234,-0.913881,-0.648513,-1.518076
406,-0.133279,-1.510044,-0.680336,-1.001235,0.296284,-0.531343,-0.823012,-0.238443,-0.802234,0.202477,1.538209,-1.518076


In [94]:
y_norm = (y_train - y_train.mean()) / y_train.std()
y_norm

,price
0,4.584970
1,3.974863
2,3.974863
3,3.954526
4,3.486777
...,...
403,-1.109362
404,-1.113430
405,-1.129699
406,-1.129699


In [95]:
# w = np.zeros(X_norm.shape[1])
# b = 0.
# print(f"Initial weights: {w}")
# print(f"Initial bias: {b}")

In [96]:
# Y_predict = X_norm.dot(w) + b
# Y_error = Y_predict - Y_norm
# cost = 1/(2*X_norm.shape[0]) * (Y_error ** 2).sum()
# dj_dw = Y_error.dot(X_norm) / X_norm.shape[0]
# dj_db = np.sum(Y_error) / X_norm.shape[0]
# w = w - 0.1 * dj_dw
# b = b - 0.1 * dj_db
# print(f'Interation: {iter + 1} Cost: {cost}')

In [108]:
def linear_regression_predict(X, w, b):
  return X.dot(w) + b

In [109]:
def linear_regression_cost(X, y, w, b, _lambda = 0):
  y_predict = linear_regression_predict(X, w, b)
  y_error = y_predict - y
  cost = ((y_error ** 2).sum() + _lambda * ((w ** 2).sum() + (b ** 2))) / (2*X.shape[0])
  return cost, y_error

In [110]:
def linear_regression_gradients(X, y, w, b, _lambda = 0):
  dj_dw = (y.dot(X) + _lambda * w) / X.shape[0]
  dj_db = (np.sum(y) + _lambda * b) / X.shape[0]
  return dj_dw, dj_db

In [111]:
def update_weights(w, b, dj_dw, dj_db, alpha = 0.1):
  w -= alpha * dj_dw
  b -= alpha * dj_db
  return w, b

In [130]:
w = np.zeros(X_norm.shape[1])
b = 0
#w = np.random.random(X_norm.shape[1])
#b = np.random.random()
print(f"Initial weights: {w}")
print(f"Initial bias: {b}")
iterations = 5000
learning_rate = 0.01
regularization_param = 10

for iter in range(iterations):
  #print(f'Iter: {iter}')
  cost, y_error = linear_regression_cost(X_norm, y_norm, w, b, _lambda = regularization_param)
  # Y_predict = X_norm.dot(w) + b
  # Y_error = Y_predict - y_norm
  # cost = (Y_error ** 2).sum() / (2*X_norm.shape[0])

  dj_dw, dj_db = linear_regression_gradients(X_norm, y_error, w, b, _lambda = regularization_param)
  # dj_dw = Y_error.dot(X_norm) / X_norm.shape[0]
  # dj_db = np.sum(Y_error) / X_norm.shape[0]

  w, b = update_weights(w, b, dj_dw, dj_db, alpha=learning_rate)
  # w -= learning_rate * dj_dw
  # b -= learning_rate * dj_db
  #print(w)
  #print(b)

  if (iter + 1) % 100 == 0:
    print(f'Interation: {iter + 1} Cost: {cost}')
  #print(f'dj_dw: {dj_dw}')
  #print(f'dj_db: {dj_db}')

print(f'w: {w}')
print(f'b: {b}')

Initial weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Initial bias: 0
Interation: 100 Cost: 0.2198420073181896
Interation: 200 Cost: 0.2104872609656977
Interation: 300 Cost: 0.2093516377341873
Interation: 400 Cost: 0.209059580701849
Interation: 500 Cost: 0.2089704861350755
Interation: 600 Cost: 0.20894105161047555
Interation: 700 Cost: 0.2089307361020291
Interation: 800 Cost: 0.2089269562690009
Interation: 900 Cost: 0.20892552569156145
Interation: 1000 Cost: 0.20892497180739894
Interation: 1100 Cost: 0.20892475399117402
Interation: 1200 Cost: 0.20892466742923263
Interation: 1300 Cost: 0.20892463278598247
Interation: 1400 Cost: 0.20892461885616445
Interation: 1500 Cost: 0.2089246132375936
Interation: 1600 Cost: 0.20892461096664908
Interation: 1700 Cost: 0.20892461004749904
Interation: 1800 Cost: 0.2089246096751368
Interation: 1900 Cost: 0.2089246095241943
Interation: 2000 Cost: 0.20892460946298255
Interation: 2100 Cost: 0.2089246094381525
Interation: 2200 Cost: 0.2089246094280785
Inter

In [70]:
print(type(y_error))
print(X_norm.shape)
dj_dw = y_error.dot(X_norm)/X_norm.shape[0]
print(dj_dw.shape)
print(dj_dw.to_numpy())

<class 'pandas.core.series.Series'>
(408, 12)
(12,)
[-8.83824604e-16  1.11022302e-16  8.94709143e-16 -4.60416019e-16
  2.06806250e-16  2.17690789e-16 -2.20411924e-16  2.06806250e-16
  4.33204670e-16 -4.44089210e-16  4.48443026e-16 -1.08845395e-16]


In [40]:
def compute_gradient(X, y, w, b):
    #print(f'{X}\n')
    #print(f'{y}\n')
    #print(f'{w}\n')
    #print(f'{b}\n')
    """
    Computes the gradient for linear regression
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters
      b (scalar)       : model parameter

    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    #print(f'{dj_dw}\n')
    dj_db = 0.

    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            #print(f'Gradient')
            #print(f'{dj_dw[j]}\n')
            #print(f'{err}\n')
            #print(f'{X[i, j]}\n')
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [41]:
print(X_norm.shape)
print(y_train.shape)
print(w.shape)
print(b)
dj_db, dj_dw = compute_gradient(X_norm.to_numpy(), y_train.to_numpy(), w, b)
print(f'dj_db: {dj_db}')
print(f'dj_dw: {dj_dw}')

(545, 12)
(545,)
(12,)
0.0
dj_db: -4766729.247706422
dj_dw: [-1000711.12691782  -684247.1374712   -966261.09057547  -785473.19197208
  -554311.7407424   -477052.72642098  -349236.09304122  -173767.70847778
  -845668.72516859  -717665.86968431  -615696.26380535  -568917.28955954]
